## Intro

This notebook generates an xlsx report providing insights into your Twingate environment from Network Events across all Remote Networks.

## Required Python Dependencies

This notebook uses dataframe and polars in order to process large network event datasets and outputs a xlsx spreadsheet, it therefore requires some very specific python libraries to work.

In [1]:
# Install python dependencies required
%pip install xlsxwriter
%pip install pandas
%pip install polars

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
    tinycss2 (>=1.1.0<1.2) ; extra == 'css'
             ~~~~~~~~^
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com

# Specify the Network Event Report to process

Change the cell below and specify the full path to the CSV extracted from your Admin Console along with the output xlsx file.

In [62]:
# specify the timezone you are in, it will become visible in the dataframe and can be used to filter activities based on localtime
local_tz = 'America/Los_Angeles'

# specify the full path to the Network Events report
full_console_report = '/Users/brendansapience/Downloads/202504_cyera_RawNetworkEvents.csv'

# specify the full path to the finalized report you'd like to create
workbookName = '/Users/brendansapience/Downloads/cyera_insight_report.xlsx'


# Load up the parsing functions below

The functions in the following cell are used to generate the final report, they can be combined and used together as well to produce more insights as needed by Admins.

The cell below only loads functions in memory.

In [63]:
import json,requests,re
import pandas as pd
from types import SimpleNamespace
import datetime, time, logging, sys, base64, math
from urllib.request import urlopen
from json import load

################################################################
# General functions used throughout the notebook
################################################################

# converts an admin console Network Event reports (csv file) to a flattened normalized dataframe
def convert_admin_console_report_to_df(conn_output,tz=None):
    df = pl.read_csv(conn_output)
    df = df.rename({"device_id":"device.id","start_time" : "timestamp.readable","status" : "connection.error_message","resource_domain" : "resource.address","applied_rule" : "resource.applied_rule","bytes_received" : "connection.rx","bytes_transferred" : "connection.tx","resource_id" : "resource.id","remote_network" : "remote_network.name","remote_network_id" : "remote_network.id","protocol" : "connection.protocol","resource_port" : "connection.resource_port","resource_ip" : "connection.resource_ip","connector_id" : "connector.id","user": "user.email", "user_id": "user.id", "client_ip" : "connection.client_ip","connector" : "connector.name"})
    df = df.rename({"relay_ips":"relay.ips","relay_ports" : "relay.ports"})
    df = df.rename({"service_account_id":"service_account.id","service_account_key_id" : "service_account_key.id"})
    df = df.rename({"service_account":"service_account.name","service_account_key" : "service_account_key.name"})
    df = df.drop(['end_time'])
    df = df.with_columns(pl.col("timestamp.readable").str.replace(r" UTC", ""))
    df = df.with_columns(pl.col('timestamp.readable').str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S%.f").dt.convert_time_zone(tz))
    df = df.with_columns(pl.format("ConnectorId:{}", "connector.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("RemoteNetworkId:{}", "remote_network.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("ResourceId:{}", "resource.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("UserId:{}", "user.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.format("DeviceId:{}", "device.id").str.encode("base64").name.keep())
    df = df.with_columns(pl.col("connection.error_message")).fill_nan('NORMAL')
    df = df.with_columns((pl.col("timestamp.readable").dt.strftime('%Y-%m-%d')).alias("timestamp.yymmdd"))
    df = df.with_columns(
        pl.col("resource.address").fill_null(pl.col("connection.resource_ip"))
    )
    return df

def get_endpoints_from_resource_activity2(df,resource):
    logging.debug("getting list of activities for a given resource.")
    adf = df.filter(pl.col("resource.applied_rule") == resource)
    ResAddresses = adf["resource.address"].unique()
    IpAddresses = adf["connection.resource_ip"].unique()
    return adf["resource.address"].unique(),adf["connection.resource_ip"].unique()

def get_serviceaccount_activity2(df,sa_name):
    logging.debug("getting all records for a given service account.")
    return df.filter(pl.col("service_account.name") == sa_name)

def get_serviceaccountkey_activity2(df,sakey_name):
    logging.debug("getting all records for a given service account key.")
    return df.filter(pl.col("service_account_key.name") == sakey_name)

def get_user_activity2(df,user):
    logging.debug("getting all records for a given user.")
    return df.filter(pl.col("user.email") == user)

def get_user_client_ips2(df,user):
    logging.debug("getting all records for a given user.")
    df1 = df.filter(pl.col("user.email") == user)
    return df1["connection.client_ip"].unique()
    
def get_errors2(df):
    return df.filter(pl.col("connection.error_message") != "NORMAL")

def get_failures2(df):
    if "event_type" in df:
        return df.filter(pl.col("event_type") == "failed_to_connect")

def get_relay_vs_p2p_traffic2(df):
    p2p_events = df.filter(pl.col("relay.ips").is_null())
    relay_events = df.filter(pl.col("relay.ips").is_not_null())
    return p2p_events,relay_events

def initiate_logging():
    root = logging.getLogger()
    root.setLevel(logging.INFO)
    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    root.addHandler(handler)

# Generate the final (XLSX) Report

You can customize the output report below in many ways:

* Change the title of each Tab
* Activate or deactivate sections of the report

In [64]:
import polars as pl
import time
import json,requests,re
from types import SimpleNamespace
import datetime, time, logging, sys, base64, math
from zoneinfo import ZoneInfo
from urllib.request import urlopen
from json import load
from xlsxwriter import Workbook

doUserIPInfo = True # Export User/IP Info
userIPSummaryTitle = "User IP Details" # Title for Workbook Tab

doUserActInfo = True # Export User Activity Details
userActTitle = "User Activity Details" # Title for Workbook Tab

doServiceAccountInfo = True
SATitle = "SA Activity Details" # Title for Workbook Tab

doServiceAccountKeyInfo = True
SAKTitle = "SA Key Activity Details" # Title for Workbook Tab

doResInfo = True # Export Full Resource List
doResTitle = "Full Resource List" # Title for Workbook Tab

doMatchList = True # Export Match List for Wildcard Resources
doMatchTitle = "Resource Matching List" # Title for Workbook Tab

doConnectorsStats = True # Export Overall Connectors Stats
doConnectorsStatsTitle = "Connector Activities" # Title for Workbook Tab

doConnectorsStatsPerConnector = True # Export Per Connectors Stats

doConnErrorsStats = True # Export Connection Error Stats
doConnErrorsStatsTitle = "Connection Errors" # Title for Workbook Tab

doDNSErrorsStats = True # Export DNS Error Stats
doDNSErrorsStatsTitle = "DNS Errors" # Title for Workbook Tab

# initiate logging
initiate_logging()
startTime = time.time()

# Convert entire report output to DataFrame for processing.
print("Converting Network Event report (csv) to a DataFrame")
df = convert_admin_console_report_to_df(full_console_report,local_tz)
print("\t => Report has been successfully converted.\n")

if (doUserIPInfo or doUserActInfo):
    unique_users = df["user.email"].unique()

    
# End User activity tab
if (doUserActInfo):
    print("Extracting Usage Data per User..")
    user_activity_data = {
        "user":[],
        "numconn":[],
        "numerr":[],
        "txbytes":[],
        "rxbytes":[],
        "p2p":[],
        "relay":[],
        "nbofdevices":[],
        "lastactivity":[]
    }
    user_activity_schema = {
        "user":pl.String, 
        "numconn":pl.Int32, 
        "numerr":pl.Int32, 
        "txbytes":pl.Int64, 
        "rxbytes": pl.Int64, 
        "p2p":pl.Float32, 
        "relay":pl.Float32,
        "nbofdevices":pl.Int32,
        "lastactivity":pl.String
    }
    
    user_activity_df = pl.DataFrame(user_activity_data, schema=user_activity_schema)
    for user in unique_users:
        if user != None:
            user_act = get_user_activity2(df,user)

            # last user activity
            lastactivity = user_act.tail(1)
            lastactivitytimestamp = lastactivity['timestamp.readable']

            # list of active user devices
            devices = user_act['device.id'].unique()
            
            # get all P2P & Relay events
            p2p,relays = get_relay_vs_p2p_traffic2(user_act)
            nb_p2p_events = len(p2p)
            nb_relay_events = len(relays)
            nb_total_events = nb_p2p_events + nb_relay_events
            if nb_total_events > 0:
                percentage_p2p = round(nb_p2p_events / nb_total_events * 100,2)
                percentage_relay = round(nb_relay_events / nb_total_events * 100,2)
            else:
                percentage_p2p = 0
                percentage_relay = 0

            # get all user errors
            user_err = get_errors2(user_act)

            # get cummulative bandwidth data            
            df_temp = user_act.with_columns(pl.col("connection.tx").cast(pl.Int64))
            total_tx = df_temp["connection.tx"].sum()
            
            df_temp = user_act.with_columns(pl.col("connection.rx").cast(pl.Int64))
            total_rx = df_temp["connection.rx"].sum()
            
            temp_act_data = {
             "user": user, 
             "numconn":len(user_act), 
             "numerr": len(user_err), 
             "txbytes":total_tx, 
             "rxbytes":total_rx, 
             "p2p": percentage_p2p, 
             "relay": percentage_relay, 
             "nbofdevices":len(devices),
             "lastactivity": lastactivitytimestamp
            }

            temp_act_df = pl.DataFrame(temp_act_data, schema=user_activity_schema)
            user_activity_df = pl.concat([user_activity_df, temp_act_df])

    user_activity_df = user_activity_df.rename(
        {
            "user":"User Email",
            "numconn" : "Nb of Connections",
            "numerr" : "Nb of Errors",
            "txbytes" : "TX (bytes)",
            "rxbytes":"RX (bytes)",
            "p2p":"P2P Connections (%)",
            "relay":"Relay Connections (%)",
            "nbofdevices":"Number of Active Devices",
            "lastactivity": "Last Activity"
        }
    )
    print("\t => done.")

# End User Public IP tab
if (doUserIPInfo):
    
    print("Extracting User and Public IPs..")

    user_ip_info_data = {
        "user":[],
        "ipinfo":[],
        "nbofips":[]
    }
    
    user_ip_info_schema = {
        "user":pl.String, 
        "ipinfo":pl.String,
        "nbofips":pl.Int32
    }

    user_ip_info_df = pl.DataFrame(user_ip_info_data, schema=user_ip_info_schema)

    for user in unique_users:
        if user != None:
            ips = get_user_client_ips2(df,user)
            iplist = []
            for ip in ips:
                iplist.append(str(ip))
                ip_info_for_user = ",".join(iplist)

            temp_ip_summary_data = {
                "user": user, 
                "ipinfo":ip_info_for_user,
                "nbofips":len(ips)
            }

            temp_ip_summary_df = pl.DataFrame(temp_ip_summary_data, schema=user_ip_info_schema)
            user_ip_info_df = pl.concat([user_ip_info_df,temp_ip_summary_df])

    user_ip_info_df = user_ip_info_df.rename(
        {
            "user":"User Email",
            "ipinfo" : "Public IPs",
            "nbofips":"Total Number of Public IPs"
        }
    )
    print("\t => done.")


# Service Account activity tab
if (doServiceAccountInfo):
    print("Extracting Usage Data per Service Account..")
    sa_activity_data = {
        "saname":[],
        "numconn":[],
        "numerr":[],
        "txbytes":[],
        "rxbytes":[],
        "p2p":[],
        "relay":[],
        "lastactivity":[]
    }
    sa_activity_schema = {
        "saname":pl.String, 
        "numconn":pl.Int32, 
        "numerr":pl.Int32, 
        "txbytes":pl.Int64, 
        "rxbytes": pl.Int64, 
        "p2p":pl.Float32, 
        "relay":pl.Float32,
        "lastactivity":pl.String
    }
    
    unique_sas = df["service_account.name"].unique()
    
    sa_activity_df = pl.DataFrame(sa_activity_data, schema=sa_activity_schema)
    for sa in unique_sas:
        if sa != None:
            sa_act = get_serviceaccount_activity2(df,sa)

            # last SA activity
            lastactivity = sa_act.tail(1)
            lastactivitytimestamp = lastactivity['timestamp.readable']

            # get all P2P & Relay events
            p2p,relays = get_relay_vs_p2p_traffic2(sa_act)
            nb_p2p_events = len(p2p)
            nb_relay_events = len(relays)
            nb_total_events = nb_p2p_events + nb_relay_events
            if nb_total_events > 0:
                percentage_p2p = round(nb_p2p_events / nb_total_events * 100,2)
                percentage_relay = round(nb_relay_events / nb_total_events * 100,2)
            else:
                percentage_p2p = 0
                percentage_relay = 0

            # get all user errors
            sa_err = get_errors2(sa_act)

            # get cummulative bandwidth data            
            df_temp = sa_act.with_columns(pl.col("connection.tx").cast(pl.Int64))
            total_tx = df_temp["connection.tx"].sum()
            
            df_temp = sa_act.with_columns(pl.col("connection.rx").cast(pl.Int64))
            total_rx = df_temp["connection.rx"].sum()
            
            temp_act_data = {
             "saname": sa, 
             "numconn":len(sa_act), 
             "numerr": len(sa_err), 
             "txbytes":total_tx, 
             "rxbytes":total_rx, 
             "p2p": percentage_p2p, 
             "relay": percentage_relay, 
             "lastactivity": lastactivitytimestamp
            }

            temp_act_df = pl.DataFrame(temp_act_data, schema=sa_activity_schema)
            sa_activity_df = pl.concat([sa_activity_df, temp_act_df])

    sa_activity_df = sa_activity_df.rename(
        {
            "saname":"Service Account Name",
            "numconn" : "Nb of Connections",
            "numerr" : "Nb of Errors",
            "txbytes" : "TX (bytes)",
            "rxbytes":"RX (bytes)",
            "p2p":"P2P Connections (%)",
            "relay":"Relay Connections (%)",
            "lastactivity": "Last Activity"
        }
    )
    print("\t => done.")


# Service Account Key activity tab
if (doServiceAccountKeyInfo):
    print("Extracting Usage Data per Service Account Key..")
    sak_activity_data = {
        "sakeyname":[],
        "saname": [], 
        "numconn":[],
        "numerr":[],
        "txbytes":[],
        "rxbytes":[],
        "p2p":[],
        "relay":[],
        "lastactivity":[]
    }
    sak_activity_schema = {
        "sakeyname":pl.String, 
        "saname":pl.String, 
        "numconn":pl.Int32, 
        "numerr":pl.Int32, 
        "txbytes":pl.Int64, 
        "rxbytes": pl.Int64, 
        "p2p":pl.Float32, 
        "relay":pl.Float32,
        "lastactivity":pl.String
    }
    
    unique_saks = df["service_account_key.name"].unique()
    
    sak_activity_df = pl.DataFrame(sak_activity_data, schema=sak_activity_schema)
    for sak in unique_saks:
        if sak != None:
            sak_act = get_serviceaccountkey_activity2(df,sak)

            # last SA activity
            lastactivity = sak_act.tail(1)
            lastactivitytimestamp = lastactivity['timestamp.readable']
            saname = lastactivity['service_account.name']

            # get all P2P & Relay events
            p2p,relays = get_relay_vs_p2p_traffic2(sak_act)
            nb_p2p_events = len(p2p)
            nb_relay_events = len(relays)
            nb_total_events = nb_p2p_events + nb_relay_events
            if nb_total_events > 0:
                percentage_p2p = round(nb_p2p_events / nb_total_events * 100,2)
                percentage_relay = round(nb_relay_events / nb_total_events * 100,2)
            else:
                percentage_p2p = 0
                percentage_relay = 0

            # get all user errors
            sak_err = get_errors2(sak_act)

            # get cummulative bandwidth data            
            df_temp = sak_act.with_columns(pl.col("connection.tx").cast(pl.Int64))
            total_tx = df_temp["connection.tx"].sum()
            
            df_temp = sak_act.with_columns(pl.col("connection.rx").cast(pl.Int64))
            total_rx = df_temp["connection.rx"].sum()
            
            temp_act_data = {
             "sakeyname": sak, 
             "saname": saname,
             "numconn":len(sak_act), 
             "numerr": len(sak_err), 
             "txbytes":total_tx, 
             "rxbytes":total_rx, 
             "p2p": percentage_p2p, 
             "relay": percentage_relay, 
             "lastactivity": lastactivitytimestamp
            }

            temp_act_df = pl.DataFrame(temp_act_data, schema=sak_activity_schema)
            sak_activity_df = pl.concat([sak_activity_df, temp_act_df])

    sak_activity_df = sak_activity_df.rename(
        {
            "sakeyname":"Service Account Key Name",
            "saname":"Service Account Name",
            "numconn" : "Nb of Connections",
            "numerr" : "Nb of Errors",
            "txbytes" : "TX (bytes)",
            "rxbytes":"RX (bytes)",
            "p2p":"P2P Connections (%)",
            "relay":"Relay Connections (%)",
            "lastactivity": "Last Activity"
        }
    )
    print("\t => done.")

    
# Overall Resource performance tab
if (doResInfo):
    print("Extracting Resource data..")
    all_errors = get_errors2(df)
    unique_addr_in_df = df["resource.address"].unique()
    res_data = {
        "address":[],
        "resourcedef":[],
        "resourceids":[],
        "rnetworks":[],
        "numconn":[],
        "numerrs":[],
        "failrate":[],
        "totaltx":[],
        "totalrx":[],
        "tcp":[],
        "udp":[],
        "icmp":[]
    }
    
    res_data_schema = {
        "address":pl.String, 
        "resourcedef":pl.String, 
        "resourceids":pl.String, 
        "rnetworks":pl.String, 
        "numconn":pl.Int32, 
        "numerrs":pl.Int32, 
        "failrate":pl.Float32, 
        "totaltx":pl.Int64, 
        "totalrx":pl.Int64, 
        "tcp":pl.String, 
        "udp":pl.String, 
        "icmp":pl.String
    }
    
    
    
    res_data_df = pl.DataFrame(res_data, schema=res_data_schema)
    
    print("Extracting Error Report..")
   
    error_data_schema = {
        "address":pl.String, 
        "resourcedef":pl.String, 
        "resourceids":pl.String, 
        "rnetworks":pl.String, 
        "numconn":pl.Int32, 
        "numerrs":pl.Int32, 
        "failrate":pl.Float32, 
        "totaltx":pl.Int64, 
        "totalrx":pl.Int64, 
        "tcp":pl.String, 
        "udp":pl.String, 
        "icmp":pl.String
    }
    
    for addr in unique_addr_in_df:
        if str(addr) != "nan":
            total_for_addr = df.filter(pl.col("resource.address") == addr)
            err_for_addr = all_errors.filter(pl.col("resource.address") == addr)

            if len(total_for_addr) > 0:
                percentage_failure = round(len(err_for_addr) / len(total_for_addr) * 100,2)
            else:
                percentage_failure = 100

            port_connected = []
            addr_activities = df.filter(pl.col("resource.address") == addr)

            # pull resource definition, resource ID (as would appear in the Admin Console & API) and Remote Network name
            res_definitions = addr_activities["resource.applied_rule"].unique()
            res_ids = addr_activities["resource.id"].unique()
            rnetwork_names = addr_activities["remote_network.name"].unique()

            res_def_list=[]
            for definition in res_definitions:
                res_def_list.append(definition)
            res_def_list_s=";".join(str(x) for x in res_def_list)
            
            res_id_list=[]
            for definition in res_ids:
                res_id_list.append(definition)
            res_id_list_s=";".join(str(x) for x in res_id_list)
            
            res_rnetwork_list=[]
            for definition in rnetwork_names:
                res_rnetwork_list.append(definition)
            res_rnetwork_list_s=";".join(str(x) for x in res_rnetwork_list)

            df_temp = addr_activities.with_columns(pl.col("connection.tx").cast(pl.Int64))
            total_tx = df_temp["connection.tx"].sum()
            
            df_temp = addr_activities.with_columns(pl.col("connection.rx").cast(pl.Int64))
            total_rx = df_temp["connection.rx"].sum()

            addr_activities_protocols = addr_activities["connection.protocol"].unique()
            addr_activities_icmp = addr_activities.filter(pl.col("connection.protocol") == "icmp")
            addr_activities_tcp = addr_activities.filter(pl.col("connection.protocol") == "tcp")
            addr_activities_udp = addr_activities.filter(pl.col("connection.protocol") == "udp")

            ports_tcp = addr_activities_tcp["connection.resource_port"].unique()
            ports_udp = addr_activities_udp["connection.resource_port"].unique()
            ports_icmp = addr_activities_icmp["connection.resource_port"].unique()

            tcp_port_list = []
            udp_port_list = []
            icmp_port_list = []
            for port in ports_tcp:
                tcp_port_list.append(port)
            for port in ports_udp:
                udp_port_list.append(port)
            for port in ports_icmp:
                icmp_port_list.append(port)
            tcp_port_list_s=";".join(str(x) for x in tcp_port_list)
            udp_port_list_s=";".join(str(x) for x in udp_port_list)
            icmp_port_list_s=";".join(str(x) for x in icmp_port_list)
            
            res_data_temp_df = pl.DataFrame(res_data, schema=error_data_schema)


            temp_res_data = {
                "address":str(addr),
                "resourcedef":str(res_def_list_s),
                "resourceids":str(res_id_list_s),
                "rnetworks":str(res_rnetwork_list_s), 
                "numconn":len(total_for_addr), 
                "numerrs":len(err_for_addr), 
                "failrate":percentage_failure, 
                "totaltx":total_tx, 
                "totalrx":total_rx, 
                "tcp":tcp_port_list_s, 
                "udp":udp_port_list_s, 
                "icmp":icmp_port_list_s
            }
            
            temp_res_df = pl.DataFrame(temp_res_data, schema=error_data_schema)
            res_data_df = pl.concat([res_data_df, temp_res_df])

    res_data_df = res_data_df.rename(
        {
            "address":"Resource Address",
            "resourcedef":"Resource Definitions",
            "resourceids":"Resource IDs",
            "rnetworks":"Remote Networks",
            "numconn" : "Nb of Connections",
            "numerrs" : "Nb of Errors",
            "totaltx" : "TX (bytes)",
            "totalrx":"RX (bytes)",
            "failrate":"Failure Rate (%)"
        }
    )
    print("\t => done.")
    
# Resource matching list
if (doMatchList):
    # get the list of unique resource definitions that have caught traffic (with the list of actual addresses hit for each resource)
    print("Extracting Resource Matching data..")
    unique_resources = df["resource.applied_rule"].unique()
    
    match_data = {
        "resource":[],
        "matches":[]
    }
    
    match_data_schema = {
        "resource":pl.String, 
        "matches":pl.String
    }
    
    match_data_df = pl.DataFrame(match_data, schema=match_data_schema)
    for r in unique_resources:
        addresses_for_resource,ips_for_resource = get_endpoints_from_resource_activity2(df,r)
        
        # if the list of unique addresses returns only 1 address and said address is None, then the source of truth should be IPs and not FQDNs
        if len(addresses_for_resource) == 1:
            addr_joined="; ".join(str(x) for x in addresses_for_resource)
            if addr_joined == "None":
                addr_joined="; ".join(str(x) for x in ips_for_resource)
        else:
            addr_joined="; ".join(str(x) for x in addresses_for_resource) 
                
        temp_match_data = {
            "resource":r, 
            "matches":addr_joined
        }
        
        temp_match_data_df = pl.DataFrame(temp_match_data, schema=match_data_schema)
        match_data_df = pl.concat([match_data_df, temp_match_data_df])

    match_data_df = match_data_df.rename(
        {
            "resource":"Resource Definition",
            "matches" : "Corresponding Addresses"
        }
    )
    print("\t => done.")

# DNS Error tab
if(doDNSErrorsStats):
    print("Extracting DNS Errors..")

    connact_data = {
            "address":[],
            "nberrors":[]
      }
    
    connact_data_schema = {
        "address":pl.String, 
        "nberrors":pl.Int64
    }

    dns_errors_df = pl.DataFrame(connact_data, schema=connact_data_schema)
    activities_dnserror = df.filter(pl.col("connection.error_message") == "DNS_ERROR")
    unique_dns_failures = activities_dnserror['resource.address'].unique()

    for addr in unique_dns_failures:
        dns_ct = activities_dnserror.filter(pl.col("resource.address") == addr).select(pl.len()).item()

        temp_dns_errors = {
            "address":addr, 
            "nberrors":dns_ct
        }
        
        temp_dns_errors_df = pl.DataFrame(temp_dns_errors, schema=connact_data_schema)

        dns_errors_df = pl.concat([dns_errors_df, temp_dns_errors_df])

    dns_errors_df = dns_errors_df.rename(
        {
            "address":"Resource Address",
            "nberrors" : "Number of DNS Errors"
        }
    )
    print("\t => done.")

# Connection Error tab
if(doConnErrorsStats):
    print("Extracting Connection Errors..")

    connact_data = {
        "address":[],
        "nberrors":[]
    }
    
    connact_data_schema = {
        "address":pl.String, 
        "nberrors":pl.Int64
    }

    conn_errors_df = pl.DataFrame(connact_data, schema=connact_data_schema)
    activities_connectionfailed = df.filter(pl.col("connection.error_message") == "CONNECTION_FAILED")
    unique_address_failures = activities_connectionfailed['resource.address'].unique()

    for addr in unique_address_failures:
        err_ct = activities_connectionfailed.filter(pl.col("resource.address") == addr).select(pl.len()).item()

        temp_conn_errors = {
            "address":addr, 
            "nberrors":err_ct
        }
        
        temp_conn_errors_df = pl.DataFrame(temp_conn_errors, schema=connact_data_schema)
        conn_errors_df = pl.concat([conn_errors_df, temp_conn_errors_df])

    conn_errors_df = conn_errors_df.rename(
        {
            "address":"Resource Address",
            "nberrors" : "Number of Connection Errors"
        }
    )
    print("\t => done.")
    
# Overall Connector performance tab
if (doConnectorsStats):
    print("Extracting Overall Connector Stats..")
    unique_connectors = df["connector.name"].unique()
    
    connact_data = {
        "connectorname":[],
        "rnname":[],
        "totalactivities":[],
        "connsuccess":[],
        "connerror":[],
        "dnserror":[]
    }
    
    connact_data_schema = {
        "connectorname":pl.String,
        "rnname":pl.String, 
        "totalactivities":pl.Int64, 
        "connsuccess":pl.Int64, 
        "connerror":pl.Int64, 
        "dnserror":pl.Int64
    }
    
    connector_data_df = pl.DataFrame(connact_data, schema=connact_data_schema)
    
    for connector in unique_connectors:
        
        conn_activities = df.filter(pl.col("connector.name") == connector)
        rn_name = conn_activities['remote_network.name'].unique()[0]
        conn_activities_normal = conn_activities.filter(pl.col("connection.error_message") == "NORMAL")
        conn_activities_connectionfailed = conn_activities.filter(pl.col("connection.error_message") == "CONNECTION_FAILED")
        conn_activities_dnserror = conn_activities.filter(pl.col("connection.error_message") == "DNS_ERROR")

        temp_res_data = {
            "connectorname":str(connector),
            "rnname":str(rn_name), 
            "totalactivities":len(conn_activities),
            "connsuccess":len(conn_activities_normal),
            "connerror":len(conn_activities_connectionfailed),
            "dnserror":len(conn_activities_dnserror)
        }
        temp_res_df = pl.DataFrame(temp_res_data, schema=connact_data_schema)

        connector_data_df = pl.concat([connector_data_df, temp_res_df])

    connector_data_df = connector_data_df.rename(
        {
            "connectorname":"Connector Name",
            "totalactivities" : "Nb of Activities",
            "rnname":"Remote Network Name",
            "connsuccess":"Successful Connections",
            "connerror":"Failed Connections",
            "dnserror":"DNS Errors"
        }
    )
    print("\t => done.")
    
# Per Connector performance tab
if (doConnectorsStatsPerConnector):
    print("Extracting Per Connector Report..")
    unique_connectors = df["connector.name"].unique()
    
    connact_data = {
        "date":[],
        "connectorname":[],
        "rnname":[],
        "totalactivities":[],
        "connsuccess":[],
        "connerror":[],
        "dnserror":[]
    }
    
    connact_data_scheme = {
        "date":pl.String,
        "connectorname":pl.String,
        "rnname":pl.String, 
        "totalactivities":pl.Int64, 
        "connsuccess":pl.Int64, 
        "connerror":pl.Int64, 
        "dnserror":pl.Int64
    }

    all_connector_data = {}
    for connector in unique_connectors:
        per_connector_data_df = pl.DataFrame(connact_data, schema=connact_data_scheme)
        conn_activities = df.filter(pl.col("connector.name") == connector)
        rn_name = conn_activities['remote_network.name'].unique()[0]
        unique_days_of_Activities = conn_activities["timestamp.yymmdd"].unique()
        for day in unique_days_of_Activities:
            conn_day_activities = conn_activities.filter(pl.col("timestamp.yymmdd") == day)
            connection_normal = conn_day_activities.filter(pl.col("connection.error_message") == "NORMAL")
            connection_errors = conn_day_activities.filter(pl.col("connection.error_message") == "CONNECTION_FAILED")
            dns_errors = conn_day_activities.filter(pl.col("connection.error_message") == "DNS_ERROR")

            temp_res_data = {
                "date":str(day),
                "connectorname":str(connector),
                "rnname":str(rn_name), 
                "totalactivities":len(conn_day_activities),
                "connsuccess":len(connection_normal),
                "connerror":len(connection_errors),
                "dnserror":len(dns_errors)
            }

            temp_res_df = pl.DataFrame(temp_res_data, schema=connact_data_scheme)

            per_connector_data_df = pl.concat([per_connector_data_df, temp_res_df])

        per_connector_data_df = per_connector_data_df.rename(
            {
                "connectorname":"Connector Name",
                "totalactivities" : "Nb of Activities",
                "rnname":"Remote Network Name",
                "connsuccess":"Successful Connections",
                "connerror":"Failed Connections",
                "dnserror":"DNS Errors"
            }
        )
        print("\t => done for Connector "+connector)

        all_connector_data[connector] = per_connector_data_df

# Generate the XLSX Report with all tabs & data
with Workbook(workbookName) as wb:

    if(doUserActInfo):
        print("Adding activity summary Tab.")
        user_activity_df.write_excel(
            workbook=wb,
            worksheet=userActTitle
        )

    if(doUserIPInfo):
        print("\nAdding IP information Tab.")
        user_ip_info_df.write_excel(
            workbook=wb,
            worksheet=userIPSummaryTitle
        )

    if(doServiceAccountInfo):
        print("Adding Service Account activity summary Tab.")
        sa_activity_df.write_excel(
            workbook=wb,
            worksheet=SATitle
        )
        
    if(doServiceAccountKeyInfo):
        print("Adding Service Account Key activity summary Tab.")
        sak_activity_df.write_excel(
            workbook=wb,
            worksheet=SAKTitle
        )
        
    if(doResInfo):
        print("Adding resource summary Tab.")
        res_data_df.write_excel(
            workbook=wb,
            worksheet=doResTitle
        )
    if(doMatchList):
        print("Adding resource match summary Tab.")
        match_data_df.write_excel(
            workbook=wb,
            worksheet=doMatchTitle
        )
    if(doDNSErrorsStats):
        print("Adding DNS error summary Tab.")
        dns_errors_df.write_excel(
            workbook=wb,
            worksheet=doDNSErrorsStatsTitle
      )

    if(doConnErrorsStats):
        print("Adding Connection error summary Tab.")
        conn_errors_df.write_excel(
            workbook=wb,
            worksheet=doConnErrorsStatsTitle
      )
    if(doConnectorsStats):
        print("Adding connector summary Tab.")
        connector_data_df.write_excel(
        workbook=wb,
        worksheet=doConnectorsStatsTitle
    )

    if(doConnectorsStatsPerConnector):
        print("Adding per Connector stats summary Tab.")
        #all_connector_data
        for key in all_connector_data:
            all_connector_data[key].write_excel(
                workbook=wb,
                worksheet=key
            )

    for WS in wb.worksheets():
        WS.autofit()
        #WS.autofilter('A1:Z1000')

    endTime = time.time()
    timeDiff = endTime - startTime
    timeDiff = timeDiff * 10**3/1000
    
    print("\nReport Generation completed. It took " + str(round(timeDiff,2)) + " seconds to generate.")

Converting Network Event report (csv) to a DataFrame
	 => Report has been successfully converted.

Extracting Usage Data per User..
	 => done.
Extracting User and Public IPs..
	 => done.
Extracting Usage Data per Service Account..
	 => done.
Extracting Usage Data per Service Account Key..
	 => done.
Extracting Resource data..
Extracting Error Report..
	 => done.
Extracting Resource Matching data..
	 => done.
Extracting DNS Errors..
	 => done.
Extracting Connection Errors..
	 => done.
Extracting Overall Connector Stats..
	 => done.
Extracting Per Connector Report..
	 => done for Connector silver-lemming
	 => done for Connector certain-tarsier
	 => done for Connector maroon-cassowary
	 => done for Connector judicious-bull
	 => done for Connector copper-eel
	 => done for Connector warping-raccoon
	 => done for Connector scrupulous-goat
	 => done for Connector cornflower-mandrill
	 => done for Connector olive-galago
	 => done for Connector shapeless-dalmatian
	 => done for Connector thistl